In [ ]:
from keras.models import load_model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

In [ ]:
from numpy import save

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

In [ ]:
import os

In [ ]:
trial_model=load_model('C:\\Users\\Om\\Desktop\\Cerberus\\Utilities\\VGGFace.h5')
WB_CNN = load_model('C:\\Users\\Om\\Desktop\\Cerberus\\Library\\Bottle_Neck.h5')

In [ ]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [ ]:
def feature_extractor(inpDir,person,timestep,sample,ex_type=True): #access directory and extract face vector
    
    
     
    if ex_type == True:
        Y_train_labels=np.ones((1,2622)) #*******
    else:
        Y_train_labels=np.zeros((1,2622))  #remove after training stage is completed  **********
        
     
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
     #input dataset most likely to be People_cerebrus/photos
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[person]
    sampleFolder = '%s\\%s' % (inpDir,personStr) #opening sample folder
    lsampleFolder = os.listdir(sampleFolder)
    print(lsampleFolder)
    sample= lsampleFolder[sample]
    
    utterFolder = '%s\\%s' % (sampleFolder,sample)
    i = 0
    lutterFolder = os.listdir(utterFolder)
    #print(utterFolder)
    #print(sample)
    #print(lutterFolder)
    
    for utterances in lutterFolder:
        print(utterances)
        utterNumber= '%s\\%s' % (utterFolder,utterances)
        lutterNumber= os.listdir(utterNumber)

        frame = lutterNumber[timestep]
        i = i + 1
        image= "%s\\%s" % (utterNumber,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))  #Dont change to 512
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)

    #print(feature_sequence)
    print("Extraction completed %d"%i)
    #print(WB_CNN.get_weights())
    #CNN_out = WB_CNN.predict(feature_sequence)  
    #print(CNN_out.shape)
    print("Prediction done")
    
    feature_sequence1 = feature_sequence[:,0,:,:]
    return feature_sequence1  

In [ ]:
#test1 = feature_extractor('C:\\Users\\Om\\Desktop\\Cerberus\\Test\\photos',0,0,1,ex_type=True)
#test1.shape

In [ ]:
def word_extractor(inpDir,person,sample,ex_type):
    timesteps=20
    count=0
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,person,i,sample,ex_type)
        count = count + 1
        if count==1 :
            CNN_out_total= CNN_out #maintaining same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
        print(str(i) + 'timestep')
        
    print('Concatenation completed')
    return CNN_out_total     

In [ ]:
def person_extractor(InpDir,person):
    p_dataset=[]
    count=0
    for i in range(3): #*********************** Iterate over each word
        count = count +1
        if person==0 and i==0:
            CNN_out_total= word_extractor(InpDir,person,i,True)
        else:
            CNN_out_total= word_extractor(InpDir,person,i,False)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dim. for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        print(str(i) + 'word')
        print('Printing Dataset')
        print(p_dataset)
    print('----------------')
    print(count)
    print(p_dataset.shape)
    return p_dataset 

In [ ]:
#test = person_extractor('C:\\Users\\Om\\Desktop\\Cerberus\\Test\\photos',0)

In [ ]:
#for i in range(60):
#    print(test[0] == test[i])

In [ ]:
def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    
    count_word = 0
    for word in range(3):
        count_i=0
        for utterance in range(1):
            count_j=0
            for timestep in range(20*word,20*(word+1),1):
                timestep = timestep + utterance
                sample_mat_j=sample_mat[timestep]
                sample_mat_j=sample_mat_j.reshape((1,2622))
                if count_j==0:
                    feature_vector=sample_mat_j
                else:
                    feature_vector=np.concatenate((feature_vector,sample_mat_j))
                count_j = count_j + 1

            if count_i==0:
                feature_mat=feature_vector
            else:
                feature_mat= np.concatenate((feature_mat,feature_vector))
            count_i= count_i + 1
            print(feature_mat.shape)
            
        if count_word==0:
            LSTM_input=feature_mat
        else:
            LSTM_input= np.concatenate((LSTM_input,feature_mat))
        count_word= count_word + 1
                
    print(LSTM_input.shape)
  
    return LSTM_input

In [ ]:
def BottleNeck(InpDir):
    p_dataset=person_extractor(InpDir,0)
    p_dataset_LSTM=reshape_LSTM(p_dataset)
    p_total_dataset=p_dataset_LSTM
    print('person done')
    return p_total_dataset

In [ ]:
#dataset = BottleNeck('C:\\Users\\Om\\Desktop\\Cerberus\\Test\\photos')

In [ ]:
#dataset1 = reshape_LSTM(dataset)
#dataset2 = dataset1.reshape(3,20,2622)
#dataset2.shape

In [ ]:
#dataset2[0] == dataset2[1]